# Agenda

1. What are decorators, and how do they work?
2. Writing a simple timing decorator
3. Using the outer function for storage

https://github.com/reuven

https://github.com/reuven/pycontw-2024-decorators%202024-PyConTW-decorators.ipynb

# DRY -- don't repeat yourself



In [1]:
def a():
    return f'a!\n'

def b():
    return f'b!\n'

print(a())
print(b())

a!

b!



In [2]:
# Fix #1: Add the lines with variables and explicit interpolation

lines = '-' * 60 + '\n'

def a():
    return f'{lines}a!\n{lines}'

def b():
    return f'{lines}b!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



In [3]:
# Fix #2: Instead of calling our function directly,
# we will pass our function to another function which
# will include the lines

lines = '-' * 60 + '\n'

def with_lines(func):
    return f'{lines}{func()}{lines}'

def a():
    return f'a!\n'

def b():
    return f'b!\n'

print(with_lines(a))
print(with_lines(b))

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



In [4]:
# Fix #3: Have with_lines return a new function,
# not a string. Calling this returned function
# will then return the string we want

lines = '-' * 60 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'a!\n'
with_lines_a = with_lines(a)

def b():
    return f'b!\n'
with_lines_b = with_lines(b)

print(with_lines_a())
print(with_lines_b())

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



In [5]:
# Fix #4: Instead of assigning to with_lines_a,
# just assign to a.  Same for with_lines_b

lines = '-' * 60 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'a!\n'
a = with_lines(a)

def b():
    return f'b!\n'
b = with_lines(b)

print(a())
print(b())

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



In [6]:
# Fix #5: Use decorator syntax

lines = '-' * 60 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines
def a():
    return f'a!\n'
# a = with_lines(a)

@with_lines
def b():
    return f'b!\n'
# b = with_lines(b)

print(a())
print(b())

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



# Summary of a decorator

1. It's a function (the decorator, what we mention with @)
2. That takes a function as an argument (the decorated function)
3. And returns a function as a result (the inner function, called wrapper)
4. That returned function (wrapper) replaces the orignal (decorated one)

# Who cares?

We now have a mechanism for hijacking a function (a) when it is defined and (b) when it is run, and we can replace either one of these stages with whatever we want.

1. Replace a function if permissions are wrong
2. Change the inputs
3. Change the outputs
4. Log information to another place
5. 

In [8]:
# let's add a new function to be decorated

lines = '-' * 60 + '\n'

def with_lines(func):
    def wrapper(*args):  # take any number of positional args
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines
def a():
    return f'a!\n'

@with_lines
def b():
    return f'b!\n'

@with_lines
def add(first, second):
    return f'{first} + {second} = {first+second}'
    
print(a())
print(b())
print(add(3, 5))

------------------------------------------------------------
a!
------------------------------------------------------------

------------------------------------------------------------
b!
------------------------------------------------------------



TypeError: with_lines.<locals>.wrapper() takes 0 positional arguments but 2 were given